In [372]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier




In [373]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [374]:
# from google.colab import drive
# drive.mount('https://drive.google.com/drive/u/0/my-drive')

In [375]:
train=pd.read_csv("/content/drive/MyDrive/Train.csv")
test=pd.read_csv('/content/drive/MyDrive/Test.csv')
#ss=pd.read_csv('SampleSubmission.csv')
#variables=pd.read_csv('VariableDefinitions.csv')


In [376]:
print('train data shape:',train.shape)

train data shape: (23524, 13)


In [377]:
print('test data shape:',test.shape)

test data shape: (10086, 12)


In [378]:
train.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [379]:
print('missing values:',train.isnull().sum())

missing values: country                   0
year                      0
uniqueid                  0
bank_account              0
location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
dtype: int64


In [380]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   bank_account            23524 non-null  object
 4   location_type           23524 non-null  object
 5   cellphone_access        23524 non-null  object
 6   household_size          23524 non-null  int64 
 7   age_of_respondent       23524 non-null  int64 
 8   gender_of_respondent    23524 non-null  object
 9   relationship_with_head  23524 non-null  object
 10  marital_status          23524 non-null  object
 11  education_level         23524 non-null  object
 12  job_type                23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB
None


In [381]:
#import preprocessing module
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Cobvert target label to numerical Data
le = LabelEncoder()
train['bank_account'] = le.fit_transform(train['bank_account'])

#Separate training features from target
X_train = train.drop(['bank_account'], axis=1)
y_train = train['bank_account']

print(y_train)

0        1
1        0
2        1
3        0
4        0
        ..
23519    0
23520    0
23521    0
23522    0
23523    0
Name: bank_account, Length: 23524, dtype: int64


In [382]:
# function to preprocess our data from train models
def preprocessing_data(data):

    # Convert the following numerical labels from interger to float
    float_array = data[["household_size", "age_of_respondent"]].values.astype(float)
    #year_convert
    year_dict = {2018:0, 2016:1, 2017:2}
    data["year"] = data.year.map(year_dict)
    # categorical features to be onverted to One Hot Encoding
    categ = ["relationship_with_head",
             "marital_status",
             "education_level",
             "job_type",
             "country"]
    
    # One Hot Encoding conversion
    data = pd.get_dummies(data, prefix_sep="_", columns=categ)
    
    # Label Encoder conversion
    data["location_type"] = le.fit_transform(data["location_type"])
    data["cellphone_access"] = le.fit_transform(data["cellphone_access"])
    data["gender_of_respondent"] = le.fit_transform(data["gender_of_respondent"])
    
    # drop uniquid column
    data = data.drop(["uniqueid"], axis=1)
    
    # scale our data into range of 0 and 1
    scaler = MinMaxScaler(feature_range=(0.9,1))
    data = scaler.fit_transform(data)
    
    return data                  

In [383]:
processed_train=preprocessing_data(X_train)
processed_test=preprocessing_data(test)

In [384]:
print(processed_train[:1])

[[0.9        0.9        1.         0.91       0.90952381 0.9
  0.9        0.9        0.9        0.9        0.9        1.
  0.9        0.9        1.         0.9        0.9        0.9
  0.9        0.9        1.         0.9        0.9        0.9
  0.9        0.9        0.9        0.9        0.9        0.9
  0.9        0.9        1.         1.         0.9        0.9
  0.9       ]]


In [385]:
print(processed_train.shape)

(23524, 37)


In [386]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split

X_Train,X_val,Y_train,Y_val=train_test_split(processed_train,y_train,stratify=y_train,
                                              test_size=0.1,random_state=42)

In [387]:
model=XGBClassifier(n_neighbors=10)

In [388]:
model.fit(X_Train,Y_train)

XGBClassifier(n_neighbors=10)

In [389]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [390]:
y_pred = model.predict(X_val)

In [391]:
print('error Rate of XGBoost:',1-accuracy_score(Y_val,y_pred))

error Rate of XGBoost: 0.11049723756906082


In [400]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Optimize model parameters
# I run this code in google colab to make the execution much faster and use the best params in the next code
param_grid = {'min_child_weighth': [1, 5, 10],
        'gamma': [0.5, 1],
        'subsample': [0.6, 0.9, 1.0],
        'max_depth': [5, 10]
        }
my_model = GridSearchCV(model, param_grid,n_jobs=-1,verbose=2,cv=5)
my_model.fit(X_Train, Y_train)
print(my_model.best_params_)        

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'gamma': 0.5, 'max_depth': 5, 'min_child_weighth': 1, 'subsample': 0.6}


In [392]:
from sklearn.metrics import confusion_matrix, accuracy_score

# fit by setting best parameters and Evaluate model
xgb_model = XGBClassifier(min_child_weight=1, gamma=1, subsample=0.9, max_depth=10)

xgb_model.fit(X_Train, Y_train)
y_pred = xgb_model.predict(X_val)

# Get error rate
print("Error rate of the XGB classifier: ", 1 - accuracy_score(Y_val, y_pred))

Error rate of the XGB classifier:  0.1079473013174671


In [393]:
# processed2 = pd.DataFrame(processed_test)
# processed2.head()

In [394]:
# print(type(processed2))

In [395]:
test.bank_account = model.predict(processed_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [396]:
# Create submission DataFrame
submission = pd.DataFrame({"uniqueid": test["uniqueid"] + " x " + test["country"],
                           "bank_account": test.bank_account})

In [397]:
# Create submission csv file csv file
submission.to_csv('SUBMISSION_11.csv', index=False)
